---
<div style="text-align:center; font-family:verdana;">
    <h1><strong>Simulación numérica del sistema de elución y
filtrado de una planta de extracción</strong></h1>
    <br>
    <br>
    <br>
    <div>
        Autor
        <h2>Juan David Argüello Plata</h2>
        Ingeniero mecánico
    </div>
    <br>
    <br>
    <br>
    <i>Tesis de maestría en investigación para optar al título de maestro en ciencias en ingeniería mecánica</i>
    <br>
    <br>
    <br>
    <br>
    <div>
        Director
        <h2>Omar Armando Gélvez Arocha</h2>
        Ingeniero mecánico <i>M.Sc.</i>
    </div>
    <br>
    <br>
    <br>
    <br>
    <strong style="font-size:140%;">Universidad Industrial de Santander</strong>
    <br>
    Facultad de ingenierías físicomecanicas
    <br>
    Escuela de ingeniería mecánica
    <br>
    Maestría en ingeniería mecánica
    <br>
    2020
</div>

---

<h2 style="text-align:center"><strong>Resumen</strong></h2>
<div style="text-align:justify">
    El proceso de extracción basado en el método de dispersión de la matriz en fase sólida, MSPD, consiste de <strong>tres etapas</strong>. La primera es la <i>etapa de pretratamiento</i>, o de molienda, en donde se busca disminuir el tamaño de partícula del material orgánico con el fin de incrementar el área de transferencia de masa. La siguiente se trata de la <i>etapa de elución y filtrado</i>, en donde se produce la extracción de metabolitos secundarios a través de un solvente; luego, se filtra el material particulado para obtener la mezcla homogénea solvente - extracto. Finalmente, se desarrolla una <i>etapa de separación de sustancias</i>, en donde se separa el solvente del extracto (producto final).
    <br>
    <br>
    Se propone una metodología de diseño automático del sistema de elución y filtrado que simule el comportamiento fluidodinámico durante la etapa de filtrado, permitiendo predecir el grado de concentración de partículas a lo largo del sistema a través de un modelo numérico basado en CFD-DEM. Esta metodología ha sido elaborada con herramientas de código abierto. Utilizando <i>Python</i> como lenguaje base, <i>Jupyter</i> como entorno de desarrollo, <i>ParaView</i> como plataforma de análisis de resultados y librerías de <i>C++</i> (como Yade, LIGGGHTS y OpenFoam) para el desarrollo de las simulaciones numéricas.
</div>

<br>

___Palabras clave:___ CFD-DEM, MSPD, Python, Jupyter, ParaView, OpenFoam, Yade.

---

# __1. Introducción__

<div style="text-align: justify">
    Los métodos numéricos son teoremas matemáticos que permiten describir la naturaleza de diferentes fenómenos de carácter físico-químico. Son ampliamente usados en ingeniería como metodologías predictivas durante el proceso de diseño funcional
y mecánico. Para el análisis de comportamientos fluidodinámicos de partículas, es común encontrar estudios que combinen los métodos numéricos de <i>elementos discretos</i> y <i>volúmenes finitos</i>, o como es mejor conocido: <i>modelo CFD-DEM</i>.
    <br>
    <br>
    El acoplamiento entre CFD-DEM se ha empleado cuando se busca desarrollar análisis de partículas y su interacción en medios viscosos. Ampliamente usado para análisis de lecho fluidizado[9], separadores de ciclón (Chu et al., 2009) y para el estudio de retención de partículas en medios filtrantes[10], por citar algunos ejemplos. Se han desarrollado estudios experimentales que corroboran la efectividad y viabilidad de las simulaciones numéricas que emplean CFD-DEM[9, 11]. 
    <br>
    <br>
    En el acoplamiento clásico entre CFD-DEM, el flujo se resuelve a través del método CFD basado en malla, mientras que la fase sólida es modelada mediante DEM para cada partícula sujeta a través de fuerzas hidrodinámicas, fuerzas de cuerpo (como la gravedad) y a través de fuerzas de contacto, actualizando valores de velocidad y posición conforme a la segunda ley de Newton (Hoomans et al., 1996; Tsuji et al., 1993; Xu y Yu, 1997). En principio, todos los métodos CFD pueden acoplarse con DEM; lo que ha dado origen a diferentes métodos discretos y continuos, tal como el método de Lattice Boltzmann (LBM), Hidrodinámica de Partículas Suaves (SPH), métodos de Diferencias Finitas y Volúmenes Finitos (FVM).
    <br>
    <br>
    Gran parte de las simulaciones reportadas en la literatura comprenden modelos 2D o sistemas prototipados de pequeña escala. En busca de acelerar los tiempos de simulación e incrementar la eficiencia computacional, se han desarrollado técnicas de computación paralela; donde gran parte de los esfuerzos han sido enfocados en la paralelización del DEM. Muchos algoritmos se han propuesto para lograr este hecho, como la técnica de espejo de dominio (Damana, et al., 2006; Washington y Meegoda, 2003), el método de subconjunto de partículas (Kafui et al., 2011) y
métodos de descomposición de dominios (Amritkar et al., 2014; Tsuji et al., 2008). El uso de estos algoritmos depende de la arquitectura del hardware. La paralelización sobre memoria compartida del sistema se alcanza, normalmente, empleando OpenMP (“Open Multi-Processing”, por sus siglas en inglés), mientras que el MPI (Interfaz de Paso de Mensajes) se emplea en sistemas de memoria distirbuida (Rabenseifner et al., 2009). Por ejemplo, Tsuji et al. (2008) paralelizaron una simulación en CFD-DEM usando MPI para el intercambio de información entre 16 CPUs, reportando el comportamiento fluidodinámico de 4.5 millones de partículas en un medio gaseoso; empleando el método unidimensional de descomposición de dominio.
</div>

## __2. Método de dispersión de la matriz en fase sólida - MSPD__

<div style="text-align: justify">
    Se trata de un método de extracción ampliamente utilizado para el estudio de muestras biológicas. Existen más de 250 publicaciones en las que se emplea este método extractivo para el análisis de extractos de distintas naturalezas [3]. Esto se debe a la alta eficiencia y bajo costo del método de extracción.
    <br>
    <br>
    El método consiste, básicamente, de tres etapas (como se puede observar en la Figura 1):
</div>

1. Maceración de la muestra con un agente dispersante (material particulado, normalmente compuesto de sílice).
2. Elución con solvente y filtrado de la mezcla solvente - extracto.
3. Separación de la mezcla solvente extracto mediante técnicas de evaporación al vacío, que permiten la reutilización del solvente en procesos de extracción posteriores.

<div align="center">
    <img src="Images/mspd.PNG" style="width: 300px;" />
</div>
<div align="center">
    <i>Figura 1.</i> Método MSPD$^{[1]}$.
</div>

### 2.1. Factores a considerar en la extracción MSPD

<div style="text-align:justify">
    Hay varios factores a considerar en la extracción MSPD, que incluye:
</div>

1. _Efecto del tamaño de partícula media:_ tamaños de partícula pequeños (entre $3 - 10 \left[ \mu m \right]$) requiere de grandes tiempos de elución y altos gradientes de presión para obtener un flujo adecuado.
2. _Agente dispersante_: el uso de silicatos infravalorados, como la arena de río, para la maceración de muestras presenta resultados diferentes a los reportados con agentes dispersantes como el $C_{18}$ o el $C_8$. A pesar de que el mismo principio de disrupción de la matriz se conserva, debido a la abrasión, es probable que se de una interacción química no deseada entre silicatos infravalorados y algunos de los flavonoides del extracto.
3. _Relación másica_: la mejor relación másica reportada en la literatura frecuenta ser una relación 1 a 4 [3], aunque puede variar de una aplicación a otra.
4. _Solvente_: el vertimiento del solvente en la columna MSPD tiene el fin de aislar analitos específicos o familias de compuestos. El tipo de solvente, y la polaridad de este, define la composición final del extracto. Existen estudios en donde se ha demostrado un incremento en el rendimiento extractivo al emplear solventes a temperaturas superiores a la temperatura ambiente e inferiores a los $60 [°C]$[8].

## __3. algo__
